# 06 - Final Pipeline (Credit Risk Model)

Bu notebook, kredi risk modeli için **uçtan uca final pipeline akışını** tek bir yerden gösterir:

1. Ham verinin okunması (`cs-training.csv`)
2. Temel temizlik ve feature engineering (`prepare_training`)
3. Eğitim / validasyon bölünmesi (80/20, stratified)
4. Final XGBoost modelinin yüklenmesi (`xgboost_credit_risk_final.pkl`)
5. Validasyon setinde performans metrikleri (ROC-AUC, Precision, Recall, F1)
6. Farklı threshold değerleri ile karar sınırının incelenmesi

Not: Aynı akış script olarak `src/pipeline.py` içinde de mevcuttur; bu notebook daha çok yöneticilere hızlıca akışı göstermek içindir.


In [1]:
# 1. Gerekli importlar ve ayarlar

import sys
from pathlib import Path
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    roc_auc_score,
    precision_score,
    recall_score,
    f1_score,
    confusion_matrix,
    precision_recall_fscore_support,
)

# Notebook içinde proje kökünü bul (cwd üzerinden)
CURRENT_DIR = Path.cwd()
if (CURRENT_DIR / "src").exists():
    PROJECT_ROOT = CURRENT_DIR
elif (CURRENT_DIR.parent / "src").exists():
    PROJECT_ROOT = CURRENT_DIR.parent
else:
    PROJECT_ROOT = CURRENT_DIR

if str(PROJECT_ROOT) not in sys.path:
    sys.path.append(str(PROJECT_ROOT))

print("Project root:", PROJECT_ROOT)
print("src var mı?:", (PROJECT_ROOT / "src").exists())

from src.config import RAW_TRAIN, FINAL_MODEL, SEED, DEFAULT_THRESHOLD
from src.data_preprocessing import prepare_training
from src.predict import _load_artifact, predict_from_df


Project root: c:\Users\YAĞMUR\Masaüstü\credit-risk-model
src var mı?: True


## 2. Veri Yükleme ve Problem Özeti

- Veri seti: **Give Me Some Credit (Kaggle)**  
- Hedef değişken: `SeriousDlqin2yrs` (müşteri 2 yıl içinde defaulta düştü mü? 0/1)  
- Sınıf dengesizliği: Default oranı düşük, bu yüzden ROC-AUC ve Recall kritik.


In [2]:
# 2. Ham veriyi yükle ve temel bilgileri göster

raw = pd.read_csv(RAW_TRAIN)
print("Ham veri shape:", raw.shape)
raw["SeriousDlqin2yrs"].value_counts(normalize=True)


Ham veri shape: (150000, 12)


SeriousDlqin2yrs
0    0.93316
1    0.06684
Name: proportion, dtype: float64

## 3. Temizlik ve Feature Engineering (`prepare_training`)

Bu adımda `src.data_preprocessing.prepare_training` fonksiyonu kullanılarak:

- Temel temizlik (age==0 düzeltmesi, eksik değer imputasyonu, delinquency cap)
- log1p dönüşümleri
- Delinquency feature'ları (TotalDelinquency, DelinquencySeverityScore, vb.)
- Risk flag'leri (EverDelinquent, Ever90DaysLate, HighDebtFlag, ...)
- Binning (AgeBin, IncomeBin, UtilizationBin, DelinqBin)
- Domain feature'lar (EffectiveDebtLoad, RealEstateExposure, FinancialStressIndex)
- Feature selection (`FINAL_DROP_COLS`)

uygulanır ve final eğitim tablosu elde edilir.


In [3]:
# 3. Temizlik + Feature Engineering uygulama

prepared = prepare_training(raw)
print("Hazırlanmış veri shape:", prepared.shape)

TARGET_COL = "SeriousDlqin2yrs"
X = prepared.drop(columns=[TARGET_COL])
y = prepared[TARGET_COL]

X.shape, y.value_counts(normalize=True)


Hazırlanmış veri shape: (150000, 27)


((150000, 26),
 SeriousDlqin2yrs
 0    0.93316
 1    0.06684
 Name: proportion, dtype: float64)

## 4. Eğitim / Validasyon Bölünmesi

Veri, %80 eğitim, %20 validasyon olacak şekilde, sınıf dengesini koruyarak (stratify) ayrılır.


In [4]:
# 4. Train / Validation split

X_train, X_val, y_train, y_val = train_test_split(
    X,
    y,
    test_size=0.2,
    random_state=SEED,
    stratify=y,
)

print("Train shape:", X_train.shape)
print("Val shape  :", X_val.shape)


Train shape: (120000, 26)
Val shape  : (30000, 26)


## 5. Final XGBoost Modelinin Yüklenmesi

`models/xgboost_credit_risk_final.pkl` içinde:

- `model` : Preprocessing + XGBoost pipeline  
- `threshold` : Business gereksinimine göre seçilmiş karar eşiği (ör. 0.81)  
- `features` : (opsiyonel) Kullanılan feature isimleri  

saklanmaktadır.


In [5]:
# 5. Model artifact'ini yükle

artifact = _load_artifact()
model = artifact["model"]
threshold = artifact.get("threshold", float(DEFAULT_THRESHOLD))

print("Yüklenen model tipi:", type(model))
print("Kullanılan threshold:", threshold)


Yüklenen model tipi: <class 'sklearn.pipeline.Pipeline'>
Kullanılan threshold: 0.81


## 6. Validasyon Setinde Performans Değerlendirmesi


In [6]:
# 6. Validasyon setinde tahmin ve metrikler

# Olasılık tahmini (default olasılığı, sınıf 1)
y_val_proba = model.predict_proba(X_val)[:, 1]

# Seçilen threshold ile 0/1 karar
y_val_pred = (y_val_proba >= threshold).astype(int)

roc = roc_auc_score(y_val, y_val_proba)
precision = precision_score(y_val, y_val_pred, zero_division=0)
recall = recall_score(y_val, y_val_pred, zero_division=0)
f1 = f1_score(y_val, y_val_pred, zero_division=0)
cm = confusion_matrix(y_val, y_val_pred)

print(f"ROC-AUC   : {roc:.4f}")
print(f"Precision : {precision:.4f}")
print(f"Recall    : {recall:.4f}")
print(f"F1-score  : {f1:.4f}")
print("Confusion Matrix (TN, FP, FN, TP):", cm.ravel())


ROC-AUC   : 0.8701
Precision : 0.4178
Recall    : 0.4753
F1-score  : 0.4447
Confusion Matrix (TN, FP, FN, TP): [26667  1328  1052   953]


In [7]:
# 7. Farklı threshold değerleri için performans 

def evaluate_threshold(th: float):
    y_pred_tmp = (y_val_proba >= th).astype(int)
    precision, recall, f1, _ = precision_recall_fscore_support(
        y_val, y_pred_tmp, average="binary", zero_division=0
    )
    positive_rate = y_pred_tmp.mean()
    print(f"Threshold: {th:.2f}")
    print(f"Pozitif oranı (yüksek riskli oranı): {positive_rate:.3f}")
    print(f"Precision: {precision:.3f}")
    print(f"Recall   : {recall:.3f}")
    print(f"F1-score : {f1:.3f}")

for th in [0.50, 0.70, float(DEFAULT_THRESHOLD), 0.90]:
    evaluate_threshold(th)
    print("-" * 40)


Threshold: 0.50
Pozitif oranı (yüksek riskli oranı): 0.246
Precision: 0.214
Recall   : 0.788
F1-score : 0.337
----------------------------------------
Threshold: 0.70
Pozitif oranı (yüksek riskli oranı): 0.115
Precision: 0.345
Recall   : 0.596
F1-score : 0.437
----------------------------------------
Threshold: 0.81
Pozitif oranı (yüksek riskli oranı): 0.076
Precision: 0.418
Recall   : 0.475
F1-score : 0.445
----------------------------------------
Threshold: 0.90
Pozitif oranı (yüksek riskli oranı): 0.036
Precision: 0.551
Recall   : 0.299
F1-score : 0.388
----------------------------------------


## 7. Final Pipeline Özeti

Bu notebook'ta kullanılan uçtan uca akış:

1. **Data Loading**  
   - `cs-training.csv` ham verisi okunur.

2. **Preprocessing & Feature Engineering**  
   - `prepare_training(df)` ile temizlik, feature engineering ve feature selection uygulanır.  
   - Sonuç: 27 feature içeren final tablo.

3. **Train / Validation Split**  
   - %80 eğitim, %20 validasyon, `SEED=42`, stratified split.

4. **Model**  
   - XGBoost tabanlı pipeline (`xgboost_credit_risk_final.pkl`).  
   - Class imbalance ve regularization ayarları `config.py` ve model parametrelerinde yönetilir.

5. **Decision Threshold**  
   - Default threshold: 0.81.  
   - Threshold, recall ihtiyacına göre ayarlanabilir (örn. riskli müşteri yakalama oranını artırmak için düşürülebilir).

6. **Business Alignment**  
   - ROC-AUC, Precision, Recall ve F1 metrikleri, bankanın risk iştahına göre değerlendirilir.

Bu notebook, `05_xgboost.ipynb` ve `src/pipeline.py` içindeki akışı **tek sayfalık** bir final pipeline görünümü olarak özetler.
